In [1]:
from datetime import datetime
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import os
pd.set_option("display.max_rows", 200)
pd.options.display.float_format = "{:,.1f}".format
float_formatter = "{:,.0f}".format
from utilities import *

In [2]:
print(os.path.join(
        OUTPUT_DIR, 'joined', 'summary', f'measure_ast007.csv'))

df = pd.read_csv(os.path.join(OUTPUT_DIR, 'joined', 'summary', f'measure_ast007.csv'))

c:\Users\higginsr\Documents\GitHub\asthma_indicators_qof\output\joined\summary\measure_ast007.csv


In [3]:
# Update labels for IMD
df.loc[(df["category"]=="imd") & (df["group"]=='1'),"group"]='1 - Most deprived'
df.loc[(df["category"]=="imd") & (df["group"]=='5'),"group"]='5 - Least deprived'
df.loc[(df["category"]=="imd") & (df["group"]=='missing'),"group"]='(Missing)'

# Update care_home flags
df.loc[(df["category"]=="care_home") & (df["group"]=='1'),"group"]='Yes'
df.loc[(df["category"]=="care_home") & (df["group"]=='0'),"group"]='No'

# Update learning_disability flags
df.loc[(df["category"]=="learning_disability") & (df["group"]=='1'),"group"]='Yes'
df.loc[(df["category"]=="learning_disability") & (df["group"]=='0'),"group"]='No'

# Update sex flags
df.loc[(df["category"]=="sex") & (df["group"]=='F'),"group"]='Female'
df.loc[(df["category"]=="sex") & (df["group"]=='M'),"group"]='Male'

# Update ethnicity flags
df.loc[(df["category"]=="sex") & (df["group"]==''),"group"]='Unknown'

In [4]:
df2 = pd.pivot_table(df[df.date == '2022-03-01'],index=['category','group'],values=['numerator','denominator', 'value'])
df2 = df2[['numerator','denominator','value']]

In [5]:
# Update order of categories in the table
df3 = df2.loc[['population','sex','age_band','ethnicity','imd','region','care_home','learning_disability']]
# Udpdate names of rows and columns
df4 = df3.rename(columns={'numerator':'Numerator','denominator':'Denominator','value':'Percentage'}
        , index={'population':'Population','sex':'Sex','age_band':'Age band','ethnicity':'Ethnicity'
            ,'imd':'IMD','region':'Region','care_home':'Care home status','learning_disability':'Record of learning disability'
            ,'F':'Female','M':'Male'})
# Remove multi index labels 
df4.index.names = [None,None]

In [6]:
# Change all NULL values from NaN to '-'
df5 = df4.style.format({'Denominator': "{:,.0f}".format
    , 'Numerator': "{:,.0f}".format
    , 'Percentage':"{0:.2%}".format}, na_rep="-")


In [8]:
#write table to an excel file (to maintain formatting), and add in percentage formatting
writer = pd.ExcelWriter(os.path.join(OUTPUT_DIR,"joined","summary","table_1.xlsx"),engine='xlsxwriter')
df5.to_excel(writer, sheet_name='Sheet1')

workbook = writer.book
worksheet = writer.sheets['Sheet1']

percent_format = workbook.add_format({'num_format':'0.00%'})
text_wrap = workbook.add_format()
text_wrap.set_text_wrap()


worksheet.set_column(4,4,11,percent_format)
worksheet.set_column(2,4,11)
worksheet.set_column(0,1,23)

writer.save()